# Construindo um container Tensorflow para o Amazon SageMaker

With Amazon SageMaker, you can package your own algorithms that can then be trained and deployed in the SageMaker environment. This notebook guides you through an example using TensorFlow that shows you how to build a Docker container for SageMaker and use it for training and inference.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies. 

Dataset: [CIFAR-10]: http://www.cs.toronto.edu/~kriz/cifar.html

## Parte 1: Empacotando e carregando seu algoritmo para uso com o Amazon SageMaker

### Overview de Docker

O Docker fornece uma maneira simples de empacotar código que é totalmente autocontida. Depois de obter uma imagem, você pode usar o Docker para executar um _container_ com base nessa imagem. Executar um container é como executar um programa na máquina, exceto que o container cria um ambiente totalmente independente para o programa ser executado. Os containers são isolados uns dos outros e do ambiente host, portanto, a forma como o programa é configurado é a maneira como ele é executado, não importa onde você o execute.

Alguns links interessantes:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### Como o Amazon SageMaker executa os containers Docker

O SageMaker trabalha com dois tipos de container `train` ou `serve`, mas também é possível usar a mesma imagem para as duas coisas.

* Nesse exemplo, não definimos um `ENTRYPOINT` no Dockerfile, então, por padrão o Docker irá executar o comando [`train` no momento de treinamento](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) e [`serve` no momento de inferência](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html). Nesse exemplo, definimos como scripts em Python, mas poderia ser qualquer programa.
* Se você especificar um `ENTRYPOINT` no Dockerfile, o programa irá ser executado no startup e terá que receber os argumentos de `train` ou `serve`.
* Se você está construindo containers diferentes para treino e inferência. No seu `ENTRYPOINT` no Dockerfile você pode ignorar o argumento não usado ou verificar.

#### Executando o container durante o treinamento

Tudo inicia no script `train`e a estrutura esperado no SageMaker é:

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### Entrada

* `/opt/ml/input/config` contém informações para controlar como seu programa é executado. `hyperparameters.json` é um dicionário formatado em JSON de nomes de hiperparâmetros para valores. Esses valores são sempre strings, portanto, pode ser necessário convertê-los. `resourceConfig.json` é um arquivo formatado em JSON que descreve o layout de rede usado para treinamento distribuído.
* `/opt/ml/input/data/<channel_name>/` contém os dados de entrada para aquele canal. Os canais são criados com base na chamada para CreateTrainingJob, mas geralmente é importante que os canais correspondam às expectativas do algoritmo. Os arquivos de cada canal são copiados do S3 para este diretório, preservando a estrutura em árvore indicada pela estrutura de chaves do S3.
* `/opt/ml/input/data/<channel_name>_<epoch_number>` é um canal para uma determinada epoch. As epochs começam do zero e aumentam um a cada vez que você as lê. Não há limite para o número de epochs que você pode executar, mas você deve fechar cada canal antes de ler a próxima epoch.

##### Saída

* `/opt/ml/model/` é onde o modelo gerado irá ser armazenado. Pode estar em qualquer formato e dividido em diferentes arquivos. O SageMaker irá empacotar o diretório e comprimir em um arquivo tar. Por fim, irá armazenar no bucket S3 especificado.
* `/opt/ml/output` diretório de escrita temporária para por exemplo escrever logs de `failure`.

#### Executando o container para inferência

Amazon SageMaker usa dois resources esperando receber as requisições:

* `/ping` recebe um `GET` para efetuar o healthcheck
* `/invocations` recebe a chamada `POST` para inferências. O formato da requisição depende de cada algoritmo

Os arquivos seguem a mesma estrutura que o treinamento.

    /opt/ml
    `-- model
        `-- <model files>



### As partes para criação do container

A pasta `container` possui todos os componentes para a criação de um container:

    .
    |-- Dockerfile
    |-- build_and_push.sh
    `-- cifar10
        |-- cifar10.py
        |-- resnet_model.py
        |-- nginx.conf
        |-- serve
        `-- train

Let's discuss each of these in turn:

* __`Dockerfile`__ descrição da criação da imagem Docker
* __`build_and_push.sh`__ script para auxiliar no build e registro
* __`cifar10`__ arquivos que estarão no container

Os arquivos na pasta cifar10:

* __`cifar10.py`__ script com implementação do algoritmo
* __`resnet_model.py`__ Resnet model
* __`nginx.conf`__ configuração do nginx
* __`serve`__ Primeiro programa executado para inferência. Somente lança o nginx e carrega o modelo
* __`train`__ Programa invocado na chamada do treinamento. Nesse caso, invoca o script cifar10.py com os hiperparâmetros do diretório /opt/ml/input/config/hyperparameters.json

### O Dockerfile

In [ ]:
!cat container/Dockerfile

### Efetuando o build e registro

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-cifar10-example

cd container

chmod +x cifar10/train
chmod +x cifar10/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## Testando localmente

### Download do dataset CIFAR-10

In [ ]:
! python utils/generate_cifar10_tfrecords.py --data-dir=/tmp/cifar-10-data

In [ ]:
# There should be three tfrecords. (eval, train, validation)
! ls /tmp/cifar-10-data

### SageMaker Python SDK para teste local

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

### Fit, Deploy, Predict

In [ ]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'local'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='sagemaker-tf-cifar10-example:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///tmp/cifar-10-data')

predictor = estimator.deploy(1, instance_type)

### Predições com o SDK em Python

In [ ]:
! pip install opencv-python

In [ ]:
import cv2
import numpy

from sagemaker.predictor import json_serializer, json_deserializer

image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))

data = {'instances': numpy.asarray(image).astype(float).tolist()}

# The request and response format is JSON for TensorFlow Serving.
# For more information: https://www.tensorflow.org/serving/api_rest#predict_api
predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)

In [ ]:
predictor.delete_endpoint()

# Part 2: Implantando no SageMaker

## Configurando o ambiente

In [ ]:
# S3 prefix
prefix = 'DEMO-tensorflow-cifar10'

In [ ]:
import sagemaker as sage

sess = sage.Session()

## Upload do dataset

In [ ]:
WORK_DIRECTORY = '/tmp/cifar-10-data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Treinando no SageMaker

Agora vamos definir nossa imagem no ECR

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'sagemaker-tf-cifar10-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'ml.m4.xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_location)

predictor = estimator.deploy(1, instance_type)

In [ ]:
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))

data = {'instances': numpy.asarray(image).astype(float).tolist()}

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

predictor.predict(data)

## Cleanup

In [ ]:
# predictor.delete_endpoint()

# Referências
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)